In [1]:
import os
import pandas as pd
import json
from pathlib import Path
import tiktoken

model = "Qwen/Qwen2.5-Coder-32B-Instruct"

# Load the raw parquet data
# parquet_path = "/share/edc/home/antonis/swe-gym-setup/verl/data/swe-bench/swe-verifier-50/train.parquet"
parquet_path = "/share/edc/home/antonis/swe-gym-setup/verl/data/swe-bench/swe-verifier-50/no-feedback/train_window.parquet"
# parquet_path = "/share/edc/home/antonis/swe-gym-setup/verl/data/swe-bench/test.parquet"
# parquet_path = "/share/edc/home/antonis/_swe-planner/moatless-tree-search//evaluations/swe-1/debug/mock_data/train_debug.parquet"
raw_data = pd.read_parquet(parquet_path)
print(f"len(raw_data): {len(raw_data)}")

df = raw_data

# Get the first row
first_row = df.iloc[0]


len(raw_data): 1452


In [4]:
first_row['extra_info']

{'action_group': 0,
 'answer': 'placeholder_answer',
 'evaluation_name': 'debug/depth_first_diff_agent/6_tests_depth_first/sampled_50_verified/no_feedback/openai/Qwen/Qwen2.5-Coder-32B-Instruct',
 'index': 0,
 'instance_id': 'django__django-12419',
 'node_id': 199,
 'question': 'Add secure default SECURE_REFERRER_POLICY / Referrer-policy header\nDescription\n\t\n#29406 added the ability for the SECURE_REFERRER_POLICY setting to set Referrer-Policy, released in Django 3.0.\nI propose we change the default for this to "same-origin" to make Django applications leak less information to third party sites.\nThe main risk of breakage here would be linked websites breaking, if they depend on verification through the Referer header. This is a pretty fragile technique since it can be spoofed.\nDocumentation: \u200bhttps://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Referrer-Policy\nThe MDN support grid is out of date: \u200bhttps://caniuse.com/#search=Referrer-Policy\n',
 'split': 'train',

In [6]:
text_str = ""

# Function to count tokens
def count_tokens(text, model_name="gpt-3.5-turbo"):
    """Count the number of tokens in a text string using tiktoken."""
    try:
        encoding = tiktoken.encoding_for_model(model_name)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")  # Fallback encoding
    
    return len(encoding.encode(text))

# Print each column's value one by one
for i, row in df.iterrows():
    for i, column in enumerate(df.columns):
        instance_id = first_row['extra_info']['instance_id']
        # print(f"column {i}, {column}")
        # print(first_row[column])
        if column == 'prompt':
            for i, p in enumerate(first_row[column]):
                prompt_str = f"--- prompt {i}, {p['role']}, {instance_id} ---"
                text_str += prompt_str
                text_str += p['content']
                text_str += "\n\n"
                
                # Count tokens in this prompt content
                token_count = count_tokens(p['content'])
                
                print(prompt_str)
                # print(p['content'])
                # print(f"Token count: {token_count}")
                print("\n\n")
    if i > 1:
        break

# Count tokens in the entire text_str
total_tokens = count_tokens(text_str)
print(f"Total tokens in all prompts: {total_tokens}")


# save text_str to a file
with open(os.path.join(os.path.dirname(parquet_path), 'text_str_fmt_summary.txt'), 'w') as f:
    f.write(text_str)

--- prompt 0, system, django__django-12419 ---



--- prompt 1, user, django__django-12419 ---



--- prompt 2, assistant, django__django-12419 ---



Total tokens in all prompts: 4847


In [7]:
print(type(first_row['prompt']))
print(type(first_row['prompt'][0]))
# print(type(first_row['prompt'][0]['content']))

<class 'numpy.ndarray'>
<class 'dict'>


In [10]:
df.iloc[0]['prompt']

array([{'content': '\n# Workflow Overview\n\n1. **Understand the Task**\n  * **Review the Task:** Carefully read the task provided in <task>.\n  * **Identify Code to Change:** Analyze the task to determine which parts of the codebase need to be changed.\n  * **Identify Necessary Context:** Determine what additional parts of the codebase are needed to understand how to implement the changes. Consider dependencies, related components, and any code that interacts with the affected areas.\n\n2. **Locate Relevant Code and Tests**\n  * **Search for Code:** Use the search functions to find relevant code if it\'s not in the current context:\n      * FindClass\n      * FindFunction\n      * FindCodeSnippet\n      * SemanticSearch\n  * **View Code:** Use ViewCode to examine necessary code spans.\n\n3. **Apply Code Changes**\n * **One Step at a Time:** You can only plan and implement one code change at a time.\n * **Indentation Requirements:**\n    * Use spaces for indentation, not tabs\n    * Ma